In [1]:
# DataFrame
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, accuracy_score
import pickle

In [3]:
# DATASET
DATASET_ENCODING = "UTF-8"

TRAIN_FILE_NAME = "train_data_prepared.csv"
TEST_FILE_NAME = "test_data_prepared.csv"

In [4]:
df = pd.read_csv(TRAIN_FILE_NAME, encoding=DATASET_ENCODING)

print(df.info())
print(df.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 10 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   sentiment         1600000 non-null  int64 
 1   ids               1600000 non-null  int64 
 2   date              1600000 non-null  object
 3   flag              1600000 non-null  object
 4   user              1600000 non-null  object
 5   text              1600000 non-null  object
 6   mapped_sentiment  1600000 non-null  object
 7   tokenized_text    1600000 non-null  object
 8   stemmed_text      1592325 non-null  object
 9   lemmatized_text   1592325 non-null  object
dtypes: int64(2), object(8)
memory usage: 122.1+ MB
None
   sentiment         ids                       date      flag  \
0          0  1467810369  2009-04-06 23:12:45-07:00  NO_QUERY   
1          0  1467810672  2009-04-06 23:12:49-07:00  NO_QUERY   
2          0  1467810917  2009-04-06 23:12:53-07:00  NO_QUE

In [5]:
df_test = pd.read_csv(TEST_FILE_NAME, encoding=DATASET_ENCODING)

print(df_test.info())
print(df_test.head(5))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 498 entries, 0 to 497
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   sentiment         498 non-null    int64 
 1   ids               498 non-null    int64 
 2   date              498 non-null    object
 3   flag              498 non-null    object
 4   user              498 non-null    object
 5   text              498 non-null    object
 6   mapped_sentiment  498 non-null    object
 7   tokenized_text    498 non-null    object
 8   stemmed_text      498 non-null    object
 9   lemmatized_text   498 non-null    object
dtypes: int64(2), object(8)
memory usage: 39.0+ KB
None
   sentiment  ids                       date     flag      user  \
0          4    3  2009-05-11 03:17:40+00:00  kindle2    tpryan   
1          4    4  2009-05-11 03:18:03+00:00  kindle2    vcu451   
2          4    5  2009-05-11 03:18:54+00:00  kindle2    chadfu   
3          4    6  

In [6]:
df.dropna(subset=['lemmatized_text'], how = 'all', inplace = True)
df_test.dropna(subset=['lemmatized_text'], how = 'all', inplace = True)

In [7]:
vectorizer = TfidfVectorizer(stop_words='english')
X_train = vectorizer.fit_transform(df.lemmatized_text)
X_test = vectorizer.transform(df_test.lemmatized_text)
y_train = df.mapped_sentiment
y_test = df_test.mapped_sentiment

In [27]:
clf = DecisionTreeClassifier(random_state=42)
clf.fit(X_train, y_train)

DecisionTreeClassifier(random_state=42)

In [30]:
with open('model.pkl','wb') as f:
    pickle.dump(clf,f)


In [41]:
y_pred = clf.predict(X_test)
# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.5301204819277109

Classification Report:
               precision    recall  f1-score   support

    NEGATIVE       0.57      0.72      0.64       177
     NEUTRAL       0.00      0.00      0.00       139
    POSITIVE       0.50      0.75      0.60       182

    accuracy                           0.53       498
   macro avg       0.36      0.49      0.41       498
weighted avg       0.38      0.53      0.44       498



C:\Users\Felix\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Felix\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Felix\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [42]:
X_test_without_neutral = vectorizer.transform(df_test[df_test['mapped_sentiment'] != 'NEUTRAL'].lemmatized_text)
y_test_without_neutral = df_test[df_test['mapped_sentiment'] != 'NEUTRAL'].mapped_sentiment

In [43]:
y_pred_without_neutral = clf.predict(X_test_without_neutral)
# Evaluate the model
print("Accuracy:", accuracy_score(y_test_without_neutral, y_pred_without_neutral))
print("\nClassification Report:\n", classification_report(y_test_without_neutral, y_pred_without_neutral))

Accuracy: 0.7353760445682451

Classification Report:
               precision    recall  f1-score   support

    NEGATIVE       0.74      0.72      0.73       177
    POSITIVE       0.73      0.75      0.74       182

    accuracy                           0.74       359
   macro avg       0.74      0.74      0.74       359
weighted avg       0.74      0.74      0.74       359



In [8]:
df.dropna(subset=['stemmed_text'], how = 'all', inplace = True)
df_test.dropna(subset=['stemmed_text'], how = 'all', inplace = True)

In [12]:
vectorizer_stemmed = TfidfVectorizer(stop_words='english')
X_train_stemmed = vectorizer_stemmed.fit_transform(df.stemmed_text)
X_test_stemmed = vectorizer_stemmed.transform(df_test.stemmed_text)
y_train_stemmed = df.mapped_sentiment
y_test_stemmed = df_test.mapped_sentiment

In [13]:
clf_stemmed = DecisionTreeClassifier(random_state=42)
clf_stemmed.fit(X_train_stemmed, y_train_stemmed)

DecisionTreeClassifier(random_state=42)

In [14]:
with open('model_stemmed.pkl','wb') as f:
    pickle.dump(clf_stemmed,f)

In [16]:
y_pred_stemmed = clf_stemmed.predict(X_test_stemmed)
# Evaluate the model
print("Accuracy:", accuracy_score(y_test_stemmed, y_pred_stemmed))
print("\nClassification Report:\n", classification_report(y_test_stemmed, y_pred_stemmed))

Accuracy: 0.5321285140562249

Classification Report:
               precision    recall  f1-score   support

    NEGATIVE       0.56      0.71      0.63       177
     NEUTRAL       0.00      0.00      0.00       139
    POSITIVE       0.51      0.77      0.61       182

    accuracy                           0.53       498
   macro avg       0.36      0.49      0.41       498
weighted avg       0.39      0.53      0.45       498



C:\Users\Felix\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Felix\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Felix\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [17]:
X_test_stemmed_without_neutral = vectorizer.transform(df_test[df_test['mapped_sentiment'] != 'NEUTRAL'].lemmatized_text)
y_test_stemmed_without_neutral = df_test[df_test['mapped_sentiment'] != 'NEUTRAL'].mapped_sentiment

In [19]:
y_pred_without_neutral = clf_stemmed.predict(X_test_stemmed_without_neutral)
# Evaluate the model
print("Accuracy:", accuracy_score(y_test_stemmed_without_neutral, y_pred_without_neutral))
print("\nClassification Report:\n", classification_report(y_test_stemmed_without_neutral, y_pred_without_neutral))

Accuracy: 0.6908077994428969

Classification Report:
               precision    recall  f1-score   support

    NEGATIVE       0.69      0.67      0.68       177
    POSITIVE       0.69      0.71      0.70       182

    accuracy                           0.69       359
   macro avg       0.69      0.69      0.69       359
weighted avg       0.69      0.69      0.69       359

